In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


In [ ]:
!git clone https://github.com/Antonello03/ExplainingHate.git

Cloning into 'ExplainingHate'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 36 (delta 7), reused 34 (delta 5), pack-reused 0
Receiving objects: 100% (36/36), 332.65 KiB | 8.32 MiB/s, done.
Resolving deltas: 100% (7/7), done.
Error downloading object: distilbert-base-uncased-finetuned-hateXplain/model.safetensors (a9ea3f7): Smudge error: Error downloading distilbert-base-uncased-finetuned-hateXplain/model.safetensors (a9ea3f70c58ba1bca7d63b6efcbcbaeb82f55b948a0f5355875ba76231c08a4d): batch response: This repository is over its data quota. Account responsible for LFS bandwidth should purchase more data packs to restore access.

Errors logged to /content/ExplainingHate/.git/lfs/logs/20240608T211708.610476267.log
Use `git lfs logs last` to view the log.
error: external filter 'git-lfs filter-process' failed
fatal: distilbert-base-uncased-finetuned-hateXplain/model.safetensors: smudge fi

In [ ]:
%cd ExplainingHate

/content/ExplainingHate


In [ ]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, pipeline
from sklearn.metrics import f1_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# HateXplain Dataset

### Content
- **Textual Data**: The dataset contains social media posts from Twitter and Gab.
- **Labels**: Each post is labeled as "normal," "offensive," or "hate," indicating the severity of the language.

### Annotations
- **Label Annotations**: Posts are annotated by multiple human annotators to ensure consistent labeling.
- **Rationales**: Annotators provide explanations highlighting specific parts of the text that influenced their labeling decision.
- **Target Communities**: Annotations include information on which communities or groups are targeted by the hate speech.

In [ ]:
dataset = load_dataset("hatexplain")
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1491: FutureWarning: The repository for hatexplain contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/hatexplain
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/15383 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1922 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1924 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'annotators', 'rationales', 'post_tokens'],
        num_rows: 15383
    })
    validation: Dataset({
        features: ['id', 'annotators', 'rationales', 'post_tokens'],
        num_rows: 1922
    })
    test: Dataset({
        features: ['id', 'annotators', 'rationales', 'post_tokens'],
        num_rows: 1924
    })
})

# Preprocessing
- I'll classify both the offensiveness in a ordinal fashion
- and the targets of the sentence

In [ ]:
X_train = dataset["train"]["post_tokens"]
X_val = dataset["validation"]["post_tokens"]
X_test = dataset["test"]["post_tokens"]

X_train = [" ".join(sublist) for sublist in X_train]
X_val = [" ".join(sublist) for sublist in X_val]
X_test = [" ".join(sublist) for sublist in X_test]

y_train_lbl = dataset["train"]["annotators"]
y_val_lbl = dataset["validation"]["annotators"]
y_test_lbl = dataset["test"]["annotators"]

y_train_lbl = [el["label"] for el in y_train_lbl]
y_val_lbl = [el["label"] for el in y_val_lbl]
y_test_lbl = [el["label"] for el in y_test_lbl]

y_train_targets = dataset["train"]["annotators"]
y_val_targets = dataset["validation"]["annotators"]
y_test_targets = dataset["test"]["annotators"]

y_train_targets = [el["target"] for el in y_train_targets]
y_val_targets = [el["target"] for el in y_val_targets]
y_test_targets = [el["target"] for el in y_test_targets]

In [ ]:
id1 = 140
id2 = 145

for el1, el2, el3 in zip(X_train[id1:id2],y_train_lbl[id1:id2],y_train_targets[id1:id2]):
    print(el1,el2,el3,sep="\n",end="\n\n")

lesson of the week from the holy book the sovereign individual once commerce flows over the internet suppression of speech becomes unstoppable devolution inevitable and the monopoly of violence by governments will end paraphrased from j d davidson rhys mogg
[1, 1, 1]
[['None'], ['None'], ['None']]

watch bbc on muslims abandoning eu <number> syria the father honestly says it bcs they do not like girls being told they have freewill by supposedly educated pple bcs women don t in islam sharia sharialaw realitycheck fgm misogyny hijab rapeculture abuse violence woman women girl girls womansrights womensrights brexit
[1, 1, 1]
[['Islam', 'Women'], ['Arab', 'Islam', 'Women'], ['Arab', 'Islam']]

<user> whatchu are <user> is a self hating homosexual who has to pay for your homosexual desires
[2, 2, 2]
[['Homosexual'], ['Homosexual'], ['Homosexual']]

the parade needed at least one severed head in honor of muslim culture
[0, 2, 0]
[['Islam'], ['Islam'], ['Islam']]

i look at the mudslime news 

I will average and normalize the label values. The actual situation is that for every sentence we have three different annotation which can take the 3 values:
- 0 - hatespeech
- 1 - normal
- 2 - offensive

I'll first reorder them, so that

- 2 = hate
- 1 = offensive
- 0 = normal

than, since for each sentence there are three different annotation, I'll average them and normalize in the range [0, 1]

(e.g., "they playing a lot of ethnic music at this white ass wedding" has labels [2, 1, 1] -> [1, 0, 0] -> 0.333 -> 0.165)

In [ ]:
lbl_map = {
    0:2,
    1:0,
    2:1
}

def avg3(ls):
    return (lbl_map[ls[0]]+lbl_map[ls[1]]+lbl_map[ls[2]])/6

y_train_lbl = [avg3(x) for x in y_train_lbl]
y_val_lbl = [avg3(x) for x in y_val_lbl]
y_test_lbl = [avg3(x) for x in y_test_lbl]

I'll also hold a categorical equivalent:
- normal  (0 -> 0.33)
- offensive (0.33 -> 0.66)
- hatespeech  (0.66 -> 1)

In [ ]:
def ordinalToCategorical (hateScore):
    if hateScore < 0.33:
        return 0
    elif hateScore < 0.66:
        return 1
    else:
        return 2

y_train_lbl_cat = [ordinalToCategorical(i) for i in y_train_lbl]
y_val_lbl_cat = [ordinalToCategorical(i) for i in y_val_lbl]
y_test_lbl_cat = [ordinalToCategorical(i) for i in y_test_lbl]
y_train_lbl_cat

[2,
 2,
 1,
 1,
 1,
 0,
 2,
 0,
 0,
 2,
 1,
 0,
 0,
 0,
 2,
 1,
 2,
 2,
 1,
 0,
 0,
 0,
 0,
 2,
 0,
 2,
 2,
 2,
 2,
 0,
 1,
 2,
 0,
 2,
 0,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 1,
 2,
 2,
 1,
 2,
 1,
 2,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 0,
 2,
 2,
 1,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 1,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 0,
 2,
 0,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 0,
 2,
 1,
 1,
 1,
 0,
 2,
 1,
 1,
 1,
 2,
 0,
 2,
 2,
 1,
 2,
 0,
 2,
 2,
 2,
 1,
 2,
 1,
 2,
 1,
 2,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 0,
 2,
 0,
 0,
 1,
 2,
 2,
 2,
 1,
 1,
 1,
 0,
 1,
 2,
 1,
 2,
 2,
 0,
 1,
 1,
 2,
 0,
 0,
 2,
 0,
 0,
 2,
 1,
 2,
 2,
 2,
 0,
 2,
 0,
 1,
 2,
 2,
 2,
 2,
 0,
 2,
 0,
 0,
 2,
 2,
 2,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 1,
 2,
 2,
 1,
 2,
 0,
 2,
 2,
 0,
 2,
 1,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 0,
 2,
 2,
 0,
 2,
 1,
 0,
 2,
 2,
 1,
 1,
 1,
 2,
 0,
 2,
 2,
 1,
 2,
 1,
 2,
 1,
 0,
 0,
 2,
 0,
 0,
 0,
 1,
 0,
 2,
 2,
 0,
 2,
 1,
 2,
 0,
 1,
 2,
 0,


A similar procedure should be done also for the targets, If two of the three annotators mentioned the same target I'll maintain it

In [ ]:
# Done by ChatGPT 4o

# Prompt:
# Suppose you have three lists of classes,
# if the same class is in at least 2 classes
# you add it to a new list to return. give me the code

from collections import Counter

def find_common_classes(list1, list2, list3):
    # Combine all lists into one
    combined_list = list1 + list2 + list3

    # Create a counter to count occurrences of each element
    counter = Counter(combined_list)

    # Create a list to store elements that appear in at least two of the lists
    result = [item for item, count in counter.items() if count >= 2]

    return result

In [ ]:
y_train_targets = [find_common_classes(trgts[0],trgts[1],trgts[2]) for trgts in y_train_targets]
y_val_targets = [find_common_classes(trgts[0],trgts[1],trgts[2]) for trgts in y_val_targets]
y_test_targets = [find_common_classes(trgts[0],trgts[1],trgts[2]) for trgts in y_test_targets]

Sample example of the changed dataset

In [ ]:
id1 = 140
id2 = 145

for sen, lbl, catlbl, targets in zip(X_train[id1:id2],y_train_lbl[id1:id2],y_train_lbl_cat[id1:id2],y_train_targets[id1:id2]):
    print(sen,lbl,f"Category: {catlbl}",targets,end="\n\n",sep="\n")

lesson of the week from the holy book the sovereign individual once commerce flows over the internet suppression of speech becomes unstoppable devolution inevitable and the monopoly of violence by governments will end paraphrased from j d davidson rhys mogg
0.0
Category: 0
['None']

watch bbc on muslims abandoning eu <number> syria the father honestly says it bcs they do not like girls being told they have freewill by supposedly educated pple bcs women don t in islam sharia sharialaw realitycheck fgm misogyny hijab rapeculture abuse violence woman women girl girls womansrights womensrights brexit
0.0
Category: 0
['Islam', 'Women', 'Arab']

<user> whatchu are <user> is a self hating homosexual who has to pay for your homosexual desires
0.5
Category: 1
['Homosexual']

the parade needed at least one severed head in honor of muslim culture
0.8333333333333334
Category: 2
['Islam']

i look at the mudslime news from europe and think i wish some goat fucker would try that shit down here in lou

In [ ]:
classes = set()
for ls in y_train_targets:
    for el in ls:
        classes.add(el)

print(classes)
print(len(classes))

{'Islam', 'Hindu', 'Other', 'Christian', 'Caucasian', 'Indigenous', 'Homosexual', 'Refugee', 'Asian', 'Hispanic', 'Economic', 'Buddhism', 'Indian', 'None', 'Disability', 'African', 'Men', 'Jewish', 'Women', 'Arab'}
20


# BERT MODEL

Since BERT wants a datasetdict I'm recostructing it with the updated data

In [ ]:
from datasets import Dataset, DatasetDict
import pandas as pd

train_data = {'text': X_train, 'label': y_train_lbl_cat}
val_data = {'text': X_val, 'label': y_val_lbl_cat}
test_data = {'text': X_test, 'label': y_test_lbl_cat}

df_train = pd.DataFrame(train_data)
df_val = pd.DataFrame(val_data)
df_test = pd.DataFrame(test_data)

train_dataset = Dataset.from_pandas(df_train)
val_dataset = Dataset.from_pandas(df_val)
test_dataset = Dataset.from_pandas(df_test)

hateXplain = DatasetDict({
    "train": train_dataset,
    "validation": val_dataset,
    "test": test_dataset
})

In [ ]:
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

hateXplain_encoded = hateXplain.map(tokenize_function, batched=True, batch_size=None)

print(hateXplain_encoded["train"][0])

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/15383 [00:00<?, ? examples/s]

Map:   0%|          | 0/1922 [00:00<?, ? examples/s]

Map:   0%|          | 0/1924 [00:00<?, ? examples/s]

{'text': 'u really think i would not have been raped by feral hindu or muslim back in india or bangladesh and a neo nazi would rape me as well just to see me cry', 'label': 2, 'input_ids': [101, 1057, 2428, 2228, 1045, 2052, 2025, 2031, 2042, 15504, 2011, 18993, 7560, 2030, 5152, 2067, 1999, 2634, 2030, 7269, 1998, 1037, 9253, 6394, 2052, 9040, 2033, 2004, 2092, 2074, 2000, 2156, 2033, 5390, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
print(model_name)
num_labels = 3
id2label = {0: "normal", 1: "offensive", 2: "hatespeech"}
label2id = {"normal": 0, "offensive": 1, "hatespeech": 2}
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels, id2label=id2label, label2id=label2id)

distilbert-base-uncased


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [ ]:
def get_F1(preds):
  preds_preds = preds.predictions[0] if isinstance(preds.predictions, tuple) else preds.predictions
  predictions = preds_preds.argmax(axis=-1)
  labels = preds.label_ids
  accuracy = f1_score(labels, predictions, average='weighted')
  return {'F1 Score': accuracy}

In [ ]:
batch_size = 16
epochs = 5
logging_steps = len(hateXplain_encoded["train"]) // batch_size
model_name_output_dir = model_name.replace("/", "-")+"-finetuned-hateXplain"
training_args = TrainingArguments(output_dir=model_name_output_dir,
                                  num_train_epochs=epochs,
                                  learning_rate=1e-4,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  weight_decay=0.01,
                                  evaluation_strategy="epoch",
                                  disable_tqdm=False,
                                  logging_steps=logging_steps,
                                  log_level="error",
                                  optim='adamw_torch'
                                  )

In [ ]:
trainer = Trainer(model=model,
                  args=training_args,
                  compute_metrics=get_F1,
                  train_dataset=hateXplain_encoded["train"],
                  eval_dataset=hateXplain_encoded["validation"],
                  tokenizer=tokenizer)
trainer.train()

In [ ]:
trainer.save_model()
trainer.evaluate()

# Uploading and testing

In [ ]:
model_path = "/content/ExplainingHate/distilbert-base-uncased-finetuned-hateXplain"

In [ ]:
from transformers import AutoModelForSequenceClassification, pipeline, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_path)

model = AutoModelForSequenceClassification.from_pretrained(model_path)
model_name = "distilbert-base-uncased"

In [ ]:
classifier = pipeline('text-classification', model=model, tokenizer=tokenizer)
print(classifier('Bruce lee worst chinese actor'))
print(classifier('i hate chinese sunflowers'))
print(classifier("i hate chinese "))

Try Others embeddings ans models
- tf-idf

trying word2vec

# Below using code of LAB 3

In [ ]:
#let's create our tokenizer function to tokenize the sentences
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
import string
punctuations = list(string.punctuation)
print(punctuations)
stopwords_list = list(stopwords.words('english'))
print(stopwords_list)

def nltk_tokenizer(sentence):
    #we lowercase all sentences
    sentence = sentence.lower()

    #here we tokenize it using nltk
    my_tokenized_tokens = word_tokenize(sentence)

    # Removing stop words and punctuations
    mytokens = [word for word in my_tokenized_tokens if word not in stopwords_list and word not in punctuations]

    # return preprocessed list of tokens
    return mytokens

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~']
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
print("apply tokenizer to test data")
df_test['tokens'] = df_test['text'].apply(nltk_tokenizer)
df_test['sentence'] =  df_test.tokens.apply(lambda x: ' '.join(x))

print("apply tokenizer to train data")
df_train['tokens'] = df_train['text'].apply(nltk_tokenizer)
df_train['sentence'] =  df_train.tokens.apply(lambda x: ' '.join(x))

apply tokenizer to test data
apply tokenizer to train data


In [ ]:
#first we need to define the vocabulary using the training data only!
vocab = set()
for sent in df_train['sentence']:
    for word in sent.split(" "):
        vocab.add(word.strip())

#print(vocab)
print(len(vocab))

24795


In [ ]:
documents_train = list(df_train.sentence)
documents_test = list(df_test.sentence)

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

#tfidf for the training data
vectorizer = CountVectorizer(lowercase=True, vocabulary=vocab)
X_count_train = vectorizer.fit_transform(documents_train)
transformer = TfidfTransformer()
X_tfidf_train = transformer.fit_transform(X_count_train)

#tfidf for the testing data
vectorizer = CountVectorizer(lowercase=True, vocabulary=vocab)
X_count_test = vectorizer.fit_transform(documents_test)
transformer = TfidfTransformer()
X_tfidf_test = transformer.fit_transform(X_count_test)

In [ ]:
Y_label_train = df_train['label'].to_list()

from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()
classifier.fit(X_tfidf_train, Y_label_train)

from sklearn import metrics

y_test = df_test['label'].to_list()
predicted = classifier.predict(X_tfidf_test)
print("Logistic Regression Accuracy:", metrics.accuracy_score(y_test, predicted))
print("Logistic Regression Precision:", metrics.precision_score(y_test, predicted, average = "weighted"))
print("Logistic Regression Recall:", metrics.recall_score(y_test, predicted, average = "weighted"))

Logistic Regression Accuracy: 0.6522869022869023
Logistic Regression Precision: 0.6353892452708894
Logistic Regression Recall: 0.6522869022869023


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Ensure the idx is an integer
        if isinstance(idx, list):
            idx = idx[0]
        embeddings, label = self.data[idx]
        embeddings = torch.tensor(embeddings, dtype=torch.float32)
        label = torch.tensor(label, dtype=torch.long)
        return embeddings, label


In [ ]:
import numpy as np
import torch
from torch.utils.data import DataLoader
import gensim.downloader as api

# Load pre-trained GloVe embeddings

embedding_dim = 50  # Size of GloVe embeddings

# Dummy sentences and labels
sentences = hateXplain_encoded["train"]["text"]
labels = hateXplain_encoded["train"]["label"]  # Example labels for a two-class problem

# Convert sentences to embeddings and pad them to the same length
max_length = max(len(sentence.split()) for sentence in sentences)

def get_word_embedding(word):
    if word in pretrained_vectors:
        return pretrained_vectors[word]
    else:
        return np.zeros(embedding_dim)  # For unknown words

def pad_sequence(embeddings, max_length, embedding_dim):
    if len(embeddings) < max_length:
        padding = np.zeros((max_length - len(embeddings), embedding_dim))
        embeddings = np.vstack((embeddings, padding))
    else:
        embeddings = embeddings[:max_length]
    return embeddings

# Create padded embedded sentences
embedded_sentences = []
for sentence in sentences:
    words = sentence.split()
    embeddings = [get_word_embedding(word) for word in words]
    padded_embeddings = pad_sequence(embeddings, max_length, embedding_dim)
    embedded_sentences.append(padded_embeddings)

# Organize data
data = list(zip(embedded_sentences, labels))

# Create dataset and dataloader
dataset = CustomDataset(data)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# Debugging:  dataloader
print("Dataset length:", len(dataset))


print("\nIterating through DataLoader:")
for batch in dataloader:
    print("Batch:", batch)
    embeddings, labels = batch
    print("Embeddings shape:", embeddings.shape)
    print("Labels:", labels)


Streaming output truncated to the last 5000 lines.
Embeddings shape: torch.Size([2, 165, 50])
Labels: tensor([2, 2])
Batch: [tensor([[[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [-0.3447,  0.6956, -0.7809,  ..., -1.3270,  0.3732,  0.0224],
         [ 0.0604,  0.3782, -0.7514,  ..., -1.2570, -0.0607,  0.4225],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

        [[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.4180,  0.2497, -0.4124,  ..., -0.1841, -0.1151, -0.7858],
         [-1.1552,  0.0248,  0.0653,  ...,  0.3401,  0.6441,  0.5512],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.

KeyboardInterrupt: 

----
# Word2Vec trained

In [ ]:
import gensim.downloader as api

pretrained_vectors = api.load('glove-wiki-gigaword-50')
emb_dim = 50

[==================================================] 100.0% 66.0/66.0MB downloaded


0        2
1        2
2        1
3        1
4        1
        ..
15378    1
15379    0
15380    0
15381    0
15382    2
Name: label, Length: 15383, dtype: int64

In [ ]:
import numpy as np
max_length = max(len(sentence) for sentence in df_train['tokens'])
print(f"max lenght: {max_length}")
counter = 0
unkown_words = []
def get_word_embedding(word):
    if word in pretrained_vectors:
        return pretrained_vectors[word]
    else:
        global counter
        global unkown_words
        unkown_words.append(word)
        counter += 1
        return np.zeros(emb_dim)  # For unknown words


def pad_sequence(embeddings, max_length, embedding_dim):
    if len(embeddings) < max_length:
        padding = np.zeros((max_length - len(embeddings), embedding_dim))
        embeddings = np.vstack((embeddings, padding))
    else:
        embeddings = embeddings[:max_length]
    return embeddings

embedded_sentences = []
for sentence in df_train['tokens']:
    words = [word for word in sentence]
    embeddings = [get_word_embedding(word) for word in words]
    padded_embeddings = pad_sequence(embeddings, max_length, emb_dim)
    embedded_sentences.append(padded_embeddings)


print(f" words not found: {counter}")
# Organize data
data = list(zip(embedded_sentences, df_train["label"]))

max lenght: 144
 words not found: 10749


In [ ]:
print(unkown_words[20:80]) #!!!! problem to be solved

['😂', '😂', 'sandnigger', '😬', 'kamleshtiwari', 'yourprophetisgay', 'fuckparties', 'dumbass', 'mudshark', 'shitposters', 'frankpost', 'lmao', 'shits', '😳', '🤔', 'cuck', 'shitheel', '😡', '😡', '😡', '😡', 'termagants', 'bestfriend', 'yanno', 'youu', '🖕', '🏽', 'pantload', 'shitskin', '\u200d', '🙂', 'beaners', 'beaners', 'kikes', '\u200d', 'dafuk', 'shiet', 'brang', 'omfg', '🤣', '\u200d', 'weevs', 'weevs', 'jidf', 'gabfam', 'naxalt', 'muslimes', 'heartattacks', 'fitty', 'shitlibs', 'refusegee', 'mudslime', 'baedophiles', 'mussie', 'snerfbucket', 'apfrica', 'zoomalis', 'comies', 'nigz', 'muzzies']


In [ ]:
len(embedded_sentences)

1924

In [ ]:
from torch.utils.data import  DataLoader
class Word2VecDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Ensure the idx is an integer

        embeddings, label = self.data[idx]
        embeddings = torch.tensor(embeddings, dtype=torch.float32)
        label = torch.tensor(label, dtype=torch.long)
        return embeddings, label

# Create dataset and dataloader


batch_size = 5
dataset = Word2VecDataset(data)
dataloader_w2v = DataLoader(dataset, batch_size=5, shuffle=True)



In [ ]:
for batch in dataloader_w2v

In [ ]:
# Convert to pytorch tensor to be used as input of the model
tfidf_tensor = torch.tensor(tfidf_matrix, dtype=torch.float32)
label_tensor = torch.tensor(y_train_lbl_cat, dtype=torch.long)

In [ ]:
from torch.utils.data import  DataLoader
class TfidfDataset(Dataset):
    def __init__(self, tfidf_tensor, labels_tensor):
        self.tfidf_tensor = tfidf_tensor
        self.labels_tensor = labels_tensor

    def __len__(self):
        return len(self.labels_tensor)

    def __getitem__(self, idx):

        return (self.tfidf_tensor[idx], self.labels_tensor[idx])

# Create dataset and dataloader
dataset = TfidfDataset(tfidf_tensor, label_tensor)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)



In [ ]:
import torch.nn as nn
import torch.optim as optim

class BiLSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(BiLSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers,
                            batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, num_classes)  # *2 for bidirectional

    def forward(self, x):
        # Set initial hidden and cell states
        h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(x.device)


        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))

        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out

# Example model parameters
input_size = emb_dim  # Size of input (embedding dimension)
hidden_size = 128
num_layers = 2
num_classes = 3

model = BiLSTMModel(input_size, hidden_size, num_layers, num_classes)


In [ ]:
print(model)

BiLSTMModel(
  (lstm): LSTM(50, 128, num_layers=2, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=256, out_features=3, bias=True)
)


In [ ]:
import numpy as np
from sklearn.metrics import precision_recall_fscore_support
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Function to calculate F1 score
def calculate_f1(preds, labels):
    preds = np.argmax(preds, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    return precision, recall, f1

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for embeddings, labels in dataloader_w2v:
        # Move tensors to the configured device
        embeddings = embeddings
        labels = labels

        # Forward pass


        outputs = model(embeddings)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

    # Validation step
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for embeddings, labels in dataloader:
            outputs = model(embeddings)
            all_preds.extend(outputs.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    precision, recall, f1 = calculate_f1(all_preds, all_labels)
    print(f'Validation Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}')


Epoch [1/10], Loss: 0.9542


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Validation Precision: 0.1573, Recall: 0.3966, F1 Score: 0.2253
Epoch [2/10], Loss: 0.9309


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Validation Precision: 0.1573, Recall: 0.3966, F1 Score: 0.2253
